A good source from... someone at Yale?

https://zoo.cs.yale.edu/classes/cs458/lectures/yfinance.html

In [184]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Data Retrieval & Basic Manipulation

In [ ]:
def fetch_stock_data(tickers, start_date, end_date, interval="60m"):
    """
    Fetch stock data for given tickers and date range.
    
    Args:
        tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT']).
        start_date (str): Start date (e.g., '2024-10-01').
        end_date (str): End date (e.g., '2024-10-31').
        interval (str): Interval for data download, valid entries: 
            ‘1m’, ‘2m’, ‘5m’, ‘15m’, ‘30m’, ‘60m’, ‘90m’, ‘1d’, ‘5d’, ‘1wk’, ‘1mo’, ‘3mo’
            recommended: 60m, 1d, 1wk, 1mo.
    
    Returns:
        dict: Dictionary with ticker keys and their data as values, and a list of tickers.
    """
    stock_data = {}

    for ticker in tickers:
        # Download data for each ticker
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval, rounding=True)
        # Store ticker_keys in dictionary
        stock_data[f"{ticker}_data"] = data
        stock_data["tickers"] = tickers
        
    return stock_data

def correct_stock_data(stock_data):
    """
    Correct stock data by removing multi-index levels and renaming index column.
    
    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
    """
    for ticker_key, dataframe in stock_data.items():
        if ticker_key == "tickers":
            continue
        stock_data[f"{ticker_key}"] = stock_data[f"{ticker_key}"].droplevel("Ticker", axis=1)
        stock_data[f"{ticker_key}"].reset_index(inplace=True)
        stock_data[f"{ticker_key}"].columns.name = f"{ticker_key.strip('_data')} Data Index"
    return stock_data


### Core Data Visualizations

In [220]:
def create_individual_graph(stock_data):
    """
    Generate line plots for the adjusted closing prices of stocks.
    
    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
    """
    for ticker, dataframe in stock_data.items():

        # Determine if markers should be shown
        markers = len(dataframe) <= 62

        # change render mode if large date range - necessary for rangebreaks? uncertain
        render_mode = "auto"
        if len(dataframe) > 999:
            render_mode = "svg"
        
        # create line chart
        fig = px.line(
            dataframe, 
            x=dataframe.columns[0], 
            y="Adj Close", 
            title=f"Price Trend for {ticker}", 
            markers = markers,                    # DISABLE FOR EXPERIMENTING W/ SCATTER
            render_mode = render_mode,
        )
        
        # remove weekends, defined as rangebreaks
        rangebreaks = [dict(bounds=["sat", "mon"])]
        
        if dataframe.columns[0] == "Datetime":
            # Exclude hours outside of trading hours
            rangebreaks.append(dict(bounds=[16, 9.5], pattern="hour"))

        # Update figure layout
        fig.update_layout(
            xaxis=dict(rangebreaks=rangebreaks),
            xaxis_title="Date/Time", 
            yaxis_title="Adjusted Close Price",            
        )
        
        fig.show()

def create_table(stock_data):
    """
    Create a display table of stock data.
    
    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
    
    Returns:
        pandas dataframe displaying desired financials, dispersed evenly by date into 12 rows
    """
    for ticker_key, dataframe in stock_data.items():
        pass

In [283]:
data = fetch_stock_data(['AAPL', 'MSFT'], '2024-10-01', '2024-10-31', interval='1d')
data2 = fetch_stock_data(['AAPL', 'MSFT'], '2024-10-01', '2024-10-31', interval='60m')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [285]:
correct_stock_data(data)
correct_stock_data(data2)

{'AAPL_data': AAPL Data Index            Datetime  Adj Close   Close    High     Low  \
 0               2024-10-01 09:30:00     224.96  224.96  229.62  224.66   
 1               2024-10-01 10:30:00     225.48  225.48  225.50  224.52   
 2               2024-10-01 11:30:00     225.31  225.31  225.94  225.05   
 3               2024-10-01 12:30:00     223.93  223.93  225.40  223.74   
 4               2024-10-01 13:30:00     225.46  225.46  225.64  223.84   
 ..                              ...        ...     ...     ...     ...   
 149             2024-10-30 11:30:00     231.20  231.20  231.66  231.06   
 150             2024-10-30 12:30:00     230.24  230.24  231.39  230.23   
 151             2024-10-30 13:30:00     230.65  230.65  231.00  229.87   
 152             2024-10-30 14:30:00     230.73  230.73  230.94  230.17   
 153             2024-10-30 15:30:00     230.14  230.14  230.84  229.56   
 
 AAPL Data Index    Open    Volume  
 0                228.75  18785859  
 1         

In [225]:
create_individual_graph(data)

In [207]:
data["AAPL_data"]

AAPL Data Index,Date,Adj Close,Close,High,Low,Open,Volume
0,2024-10-01,225.96,226.21,229.65,223.74,229.52,63285000
1,2024-10-02,226.53,226.78,227.37,223.02,225.89,32880600
2,2024-10-03,225.42,225.67,226.81,223.32,225.14,34044200
3,2024-10-04,226.55,226.80,228.00,224.13,227.90,37245100
4,2024-10-07,221.45,221.69,225.69,221.33,224.50,39505400
5,2024-10-08,225.52,225.77,225.98,223.25,224.30,31855700
6,2024-10-09,229.29,229.54,229.75,224.83,225.23,33591100
7,2024-10-10,228.79,229.04,229.50,227.17,227.78,28183500
8,2024-10-11,227.30,227.55,229.41,227.34,229.30,31759200
9,2024-10-14,231.05,231.30,231.73,228.60,228.70,39882100


In [187]:
ticker = yf.Ticker("AAPL")

news = ticker.get_news()
news_df = pd.DataFrame(news)
news_df

,uuid,title,publisher,link,providerPublishTime,type,thumbnail,relatedTickers
0,4c6467c5-1862-376b-a3a6-193a87c2c6b8,Saxo Bank’s Outrageous 2025 Prediction for Nvi...,Insider Monkey,https://finance.yahoo.com/news/saxo-bank-outra...,1733254781,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,"[NVDA, AAPL]"
1,4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b,"Magnificent Seven Stocks: Tesla Slides, While ...",Investor's Business Daily,https://finance.yahoo.com/m/4205eaa9-f620-3a0b...,1733254242,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,"[NVDA, AAPL, TSLA, META, MSFT]"
2,9232f1da-49dc-3846-9333-f4d86d1fa63b,"A current Apple employee is suing the company,...",Fortune,https://finance.yahoo.com/news/current-apple-e...,1733253424,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[AAPL]
3,0485160c-4da6-3f58-a057-bbc81ff89701,Apple (AAPL)’s AI Ambitions: Loop’s ‘Buy’ Rati...,Insider Monkey,https://finance.yahoo.com/news/apple-aapl-ai-a...,1733251840,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[AAPL]
4,ce2ffe06-f882-3e0d-8b97-fa77d137e8bb,Warren Buffett Stocks: What's Inside Berkshire...,Investor's Business Daily,https://finance.yahoo.com/m/ce2ffe06-f882-3e0d...,1733250975,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,"[BRK-A, AAPL]"
5,b3f1533e-a4ae-308c-abe7-38af65187e3a,Apple's Multi-Year iPhone Cycle And App Store ...,Benzinga,https://finance.yahoo.com/news/apples-multi-ip...,1733250691,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[AAPL]
6,de8f9c30-0ecf-33d7-9a65-4da9b624adbd,Apple Stock Keeps Climbing. This Analyst Says ...,Barrons.com,https://finance.yahoo.com/m/de8f9c30-0ecf-33d7...,1733249940,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[AAPL]
7,cbfe6d7e-1012-3e5f-a6af-74863087d1a8,Apple Faces Lawsuit for Allegedly Spying on Em...,GuruFocus.com,https://finance.yahoo.com/news/apple-faces-law...,1733244703,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[AAPL]


In [188]:
dir(ticker)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_analysis',
 '_data',
 '_download_options',
 '_earnings',
 '_earnings_dates',
 '_expirations',
 '_fast_info',
 '_fetch_ticker_tz',
 '_financials',
 '_fundamentals',
 '_funds_data',
 '_get_ticker_tz',
 '_holders',
 '_isin',
 '_lazy_load_price_history',
 '_news',
 '_options2df',
 '_price_history',
 '_quote',
 '_shares',
 '_tz',
 '_underlying',
 'actions',
 'analyst_price_targets',
 'balance_sheet',
 'balancesheet',
 'basic_info',
 'calendar',
 'capital_gains',
 'cash_flow',
 'cashflow',
 'dividends',
 'earnings',
 'earnings_dates',
 'earnings_estimate',
 'earnings_history',
 'eps_revisions',
 'eps_trend',


In [189]:
for key, value in ticker.info.items():
    print(f"{key}: {value}")

address1: One Apple Park Way
city: Cupertino
state: CA
zip: 95014
country: United States
phone: (408) 996-1010
website: https://www.apple.com
industry: Consumer Electronics
industryKey: consumer-electronics
industryDisp: Consumer Electronics
sector: Technology
sectorKey: technology
sectorDisp: Technology
longBusinessSummary: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangement

In [190]:
# PE ratio will look something like this
ticker.info["previousClose"] / (ticker.info["netIncomeToCommon"] / ticker.info["sharesOutstanding"])

39.12956282740667

In [191]:
ticker.info["dividendYield"]

0.0041

In [192]:
ticker.info["beta"]

1.24

In [193]:
ticker = yf.Ticker("AAPL")
history = ticker.history(period="5y")
history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-12-05 00:00:00-05:00,63.934952,64.443931,63.678039,64.368790,74424400,0.0,0.0
2019-12-06 00:00:00-05:00,64.829292,65.682434,64.785660,65.612144,106075600,0.0,0.0
2019-12-09 00:00:00-05:00,65.440071,65.633965,64.206406,64.693573,128042400,0.0,0.0
2019-12-10 00:00:00-05:00,65.100761,65.457047,64.436661,65.071678,90420400,0.0,0.0
2019-12-11 00:00:00-05:00,65.151634,65.706665,65.076500,65.626678,78756800,0.0,0.0
...,...,...,...,...,...,...,...
2024-11-27 00:00:00-05:00,234.470001,235.690002,233.809998,234.929993,33498400,0.0,0.0
2024-11-29 00:00:00-05:00,234.809998,237.809998,233.970001,237.330002,28481400,0.0,0.0
2024-12-02 00:00:00-05:00,237.270004,240.789993,237.160004,239.589996,48137100,0.0,0.0


In [194]:
ticker.major_holders

Breakdown,Value
insidersPercentHeld,0.02056
institutionsPercentHeld,0.61903
institutionsFloatPercentHeld,0.63203
institutionsCount,6485.00000


In [195]:
ticker.institutional_holders

,Date Reported,Holder,pctHeld,Shares,Value
0,2023-06-30,Vanguard Group Inc,0.0834,1303688506,252876459508
1,2023-06-30,Blackrock Inc.,0.0665,1039640859,201659137420
2,2023-06-30,"Berkshire Hathaway, Inc",0.0586,915560382,177591247296
3,2023-06-30,State Street Corporation,0.0370,578897858,112288817516
4,2023-06-30,"FMR, LLC",0.0196,307066638,59561715772
5,2023-06-30,"Geode Capital Management, LLC",0.0186,291538165,56549657865
6,2023-06-30,Price (T.Rowe) Associates Inc,0.0145,226650943,43963483413
7,2023-06-30,Morgan Stanley,0.0131,204714950,39708558851
8,2022-12-31,Norges Bank Investment Management,0.0107,167374278,21746939940
9,2023-06-30,Northern Trust Corporation,0.0105,164536073,31915062079


In [196]:
ticker.calendar

{'Dividend Date': datetime.date(2024, 11, 13),
 'Ex-Dividend Date': datetime.date(2024, 11, 7),
 'Earnings Date': [datetime.date(2025, 1, 30), datetime.date(2025, 2, 3)],
 'Earnings High': 2.5,
 'Earnings Low': 2.19,
 'Earnings Average': 2.35487,
 'Revenue High': 129887000000,
 'Revenue Low': 119563000000,
 'Revenue Average': 124365081850}

In [197]:
ticker.recommendations

,period,strongBuy,buy,hold,sell,strongSell
0,0m,8,24,12,1,2
1,-1m,8,24,12,1,2
2,-2m,8,23,12,1,2
3,-3m,8,24,12,0,2


In [198]:
ticker.sustainability

,esgScores
maxAge,86400
totalEsg,16.79
environmentScore,0.55
socialScore,7.52
governanceScore,8.72
ratingYear,2024
ratingMonth,11
highestControversy,3.0
peerCount,238
esgPerformance,LAG_PERF
